# FIRST EXPERIMENTATION - FIRST ASSUMPTION, WITHOUT LABELING CORRECTION

## Training a sentiment analysis classifier based on supervised machine learning algorithms

In [77]:
import string

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score

In [78]:
pd.set_option('display.max_colwidth', None)

In [79]:
stop_words = set(stopwords.words('spanish'))

In [80]:
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

### Loading labeled tweets

In [81]:
# Dataset loaded from: https://docs.google.com/spreadsheets/d/11_E2ngsEOyTQwbwVVRHY5urzFF95BQCV/edit#gid=1788161364
tweets_df = pd.read_csv('./data/tweets_labeled.csv', sep = ',')

In [82]:
tweets_df.shape

(296, 2)

In [83]:
tweets_df.head()

,full_text,sentiment
0,@Nata_Salud @Sandrag69 @AndresMejiaV ¡Hola Natalia! Te invitamos a descubrir tu #MatchPresidencial aquí: https://t.co/0E1tZKypTK,neutral
1,@supershadai @Registraduria Quien o que institución en Colombia atiende los reclamos al fraude electoral?\r\nPorque no suspender a al registrador que ya la defeco en las elecciones de senado y camara.\r\nHay una desconfianza general en cuanto a las presidenciales.\r\nEstán provocando una respuesta violenta.,negative
2,@BOLIBAR2 @AndresPastrana_ @santiagoangelp Un poco tarde con las elecciones encima… mal para Colombia,negative
3,"No encontraron otra alternativa que llenar de miedo a Colombia, utilizan sus paramilitares para ganar elecciones. Es ahora o nunca @petrogustavo",positive
4,"@BOLIBAR2 @CNE_COLOMBIA @AndresPastrana_ Aquí no va a pasar nada, y petro de va a robar las elecciones presidenciales y el país",negative


In [84]:
df2=tweets_df.replace("neutral","positive/neutral")

In [85]:
df2=df2.replace("positive","positive/neutral")

In [86]:
df2.head()

,full_text,sentiment
0,@Nata_Salud @Sandrag69 @AndresMejiaV ¡Hola Natalia! Te invitamos a descubrir tu #MatchPresidencial aquí: https://t.co/0E1tZKypTK,positive/neutral
1,@supershadai @Registraduria Quien o que institución en Colombia atiende los reclamos al fraude electoral?\r\nPorque no suspender a al registrador que ya la defeco en las elecciones de senado y camara.\r\nHay una desconfianza general en cuanto a las presidenciales.\r\nEstán provocando una respuesta violenta.,negative
2,@BOLIBAR2 @AndresPastrana_ @santiagoangelp Un poco tarde con las elecciones encima… mal para Colombia,negative
3,"No encontraron otra alternativa que llenar de miedo a Colombia, utilizan sus paramilitares para ganar elecciones. Es ahora o nunca @petrogustavo",positive/neutral
4,"@BOLIBAR2 @CNE_COLOMBIA @AndresPastrana_ Aquí no va a pasar nada, y petro de va a robar las elecciones presidenciales y el país",negative


In [87]:
df2['sentiment'].value_counts(dropna = False, normalize = True)

negative            0.668919
positive/neutral    0.331081
Name: sentiment, dtype: float64

### Leaving out unlabeled texts, this data is not useful for training or validating a supervised model

In [88]:
# Removing  unlabeled tweets
tweets_labeled_df = df2.loc[tweets_df['sentiment'].notnull()]

In [89]:
tweets_labeled_df.shape

(296, 2)

In [90]:
tweets_unlabeled_df = df2.loc[df2['sentiment'].isnull()]

In [91]:
tweets_unlabeled_df.shape

(0, 2)

In [92]:
# Scenario 3: Working with 2 classes
tweets_labeled_df['sentiment'] = tweets_labeled_df['sentiment']

### Splitting train and test datasets

In [93]:
X_train, X_test, y_train, y_test = train_test_split(tweets_labeled_df['full_text'], tweets_labeled_df['sentiment'], test_size = 0.2, stratify = tweets_labeled_df['sentiment'], random_state = 1)


In [94]:
X_train.shape

(236,)

In [95]:
pd.Series(y_train).value_counts(normalize = True)

negative            0.669492
positive/neutral    0.330508
Name: sentiment, dtype: float64

In [96]:
X_test.shape

(60,)

In [97]:
pd.Series(y_test).value_counts(normalize = True)

negative            0.666667
positive/neutral    0.333333
Name: sentiment, dtype: float64

### Vectorizing texts

<table>
    <tbody>
        <tr>
            <td>
                <h4>Bag of Words</h4>
                <img src="imgs/bow.png" style="width: 500px;">
            </td>
            <td>
                <h4>TF-IDF</h4>
                <img src="imgs/tf-idf.png" style="width: 500px;">
            </td>
        </tr>
    </tbody>
</table>

In [98]:
bow = CountVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [99]:
tfidf = TfidfVectorizer(tokenizer = tokenizer, stop_words = stop_words)

In [100]:
X_bow = bow.fit_transform(X_train)

In [101]:
X_tfidf = tfidf.fit_transform(X_train)

### Training and evaluating a model using BOW

In [102]:
model = RandomForestClassifier()

In [103]:
model.fit(X_bow, y_train)

RandomForestClassifier()

In [104]:
y_train_bow_predict = model.predict(X_bow)
y_test_bow_predict = model.predict(bow.transform(X_test))

In [105]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_bow_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [106]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_bow_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [107]:
# Metrics calculation for more than two classes
print('Precision:', precision_score(y_test, y_test_bow_predict, average = None))
print('Recall:', recall_score(y_test, y_test_bow_predict, average = None))
print('F1:', f1_score(y_test, y_test_bow_predict, average = None))

Precision: [0.8 1. ]
Recall: [1.  0.5]
F1: [0.88888889 0.66666667]


### Training and evaluating a model using TF-IDF

In [108]:
model = RandomForestClassifier()

In [109]:
model.fit(X_tfidf, y_train)

RandomForestClassifier()

In [110]:
y_train_tfidf_predict = model.predict(X_tfidf)
y_test_tfidf_predict = model.predict(bow.transform(X_test))

In [111]:
ConfusionMatrixDisplay.from_predictions(y_train, y_train_tfidf_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [112]:
ConfusionMatrixDisplay.from_predictions(y_test, y_test_tfidf_predict)

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'

In [113]:
# Metrics calculation for more than two classes
print('Precision:', precision_score(y_test, y_test_tfidf_predict, average = None))
print('Recall:', recall_score(y_test, y_test_tfidf_predict, average = None))
print('F1:', f1_score(y_test, y_test_tfidf_predict, average = None))

Precision: [0.88095238 0.83333333]
Recall: [0.925 0.75 ]
F1: [0.90243902 0.78947368]


### How interpret the results?

### Analyzing errors Bag of Words

In [115]:
error_df1 = pd.concat(
    [ pd.concat([X_test, y_test ], axis = 1).reset_index(),
    pd.Series(y_test_bow_predict) ]
, axis = 1).rename(columns = { 'sentiment': 'actual', 0: 'predicted' })

error_df1.drop('index', inplace = True, axis = 1)

In [116]:
error_df1.shape

(60, 3)

In [117]:
error_df1.loc[error_df1['actual'] != error_df1['predicted']].head(20)

,full_text,actual,predicted
7,"¿Y Si gana @petrogustavo? ¡Colombia 🇨🇴escucha!\r\n\r\nOtro clip de la emisión de #EncuentroLibertario🗽, con @CarlosAChacon, del @ICPColombia, y a @Juan__Angel__, del @libertariocol sobre las elecciones presidenciales colombianas.\r\n\r\nVea el video completo aquí: https://t.co/FFZeuyo62H https://t.co/Lyvy7pOnnd",positive/neutral,negative
17,Señores. @PGN_COL\r\n\r\nEl país reclama elecciones transparentes en primera vuelta presidencial el 29 de mayo.\r\n\r\nOpinión. @PGN_COL @CNE_COLOMBIA @moecolombia https://t.co/ceSkMa0f3W,positive/neutral,negative
19,"El Consejo Nacional Electoral @CNE_COLOMBIA envió concepto jurídico al Tribunal de Cundinamarca en el que pide que no se suspenda al registrador Alexander Vega. \r\n\r\nSegún el organismo electoral, las elecciones legislativas cumplieron con todas las garantías de transparencia.",positive/neutral,negative
21,"El Tribunal Administrativo de Cundinamarca negó la solicitud de suspensión provisional del Registrador Nacional, Alexander Vega. Sin embargo, decretó unas medidas cautelares para garantizar las elecciones. | A 23 días para la elección presidencial en Colombia, opine. https://t.co/KNA8oitIEa",positive/neutral,negative
22,(Elecciones Colombia: ¿qué muestran las más recientes encuestas presidenciales?) publicado en https://t.co/XJ7xIbDAXM - https://t.co/kBmI6ABhaM https://t.co/iDU3rbQpNu,positive/neutral,negative
41,"🇨🇴 ¿Qué pasaría en un ballotage entre @petrogustavo\r\ny @FicoGutierrez?\r\n\r\n🔍 A un mes de las elecciones que definirán el futuro del país, le preguntamos a 3 mil colombianos y colombianas y este es el resultado 👇\r\n\r\nhttps://t.co/Gsqqol98zk https://t.co/QDX164ZE9U",positive/neutral,negative
44,#Elecciones2022\r\nEn Comité de Seguimiento Electoral de cara a las elecciones del 29 de mayo nuevas conclusiones y peticiones. @MagdalenaGober\r\n@SantaMartaDTCH\r\n@Registraduria\r\n@CNE_COLOMBIA\r\n👇\r\n“La Democracia es un compromiso de todos”: @rmontoyai Infante https://t.co/f3XcwEB599,positive/neutral,negative
45,"@GustavoBolivar Debieran inventar algo diferente están igual que Chávez y Maduro cada vez que vienen elecciones inventaba guerras con Colombia, que había intereses en matarlo y ustedes igualito inventan enfermedades, atentados etc, etc.",positive/neutral,negative
52,"Episodio 8 - En este nuevo episodio Alicia Eugenia Silva nos plantea una reflexión, en estas elecciones ¿el país sigue polarizado? o ¿nace un nuevo bipartidismo en Colombia? https://t.co/HYnK26DpNn via @YouTube",positive/neutral,negative
57,Cerca de 3 mil jurados de votación de la Registraduría están inhabilitados. \r\n\r\nLa indagación fue adelantada por la Procuraduría ante las primarias de las elecciones presidenciales de Colombia. \r\nhttps://t.co/DrcoKxOTPs,positive/neutral,negative


### Analyzing errors TF-IDF

In [118]:
error_df2 = pd.concat(
    [ pd.concat([X_test, y_test ], axis = 1).reset_index(),
    pd.Series(y_test_tfidf_predict) ]
, axis = 1).rename(columns = { 'sentiment': 'actual', 0: 'predicted' })

error_df2.drop('index', inplace = True, axis = 1)

In [119]:
error_df2.shape

(60, 3)

In [120]:
error_df2.loc[error_df2['actual'] != error_df2['predicted']].head(20)

,full_text,actual,predicted
8,Petro promete reanudar relaciones diplomáticas con Maduro si gana elecciones en Colombia\r\n#TalCual #ClaroyRaspao #AmigosDeNuevo \r\nhttps://t.co/Kpm7VZydwZ https://t.co/xyy0uPwwzZ,negative,positive/neutral
19,"El Consejo Nacional Electoral @CNE_COLOMBIA envió concepto jurídico al Tribunal de Cundinamarca en el que pide que no se suspenda al registrador Alexander Vega. \r\n\r\nSegún el organismo electoral, las elecciones legislativas cumplieron con todas las garantías de transparencia.",positive/neutral,negative
21,"El Tribunal Administrativo de Cundinamarca negó la solicitud de suspensión provisional del Registrador Nacional, Alexander Vega. Sin embargo, decretó unas medidas cautelares para garantizar las elecciones. | A 23 días para la elección presidencial en Colombia, opine. https://t.co/KNA8oitIEa",positive/neutral,negative
27,"Así reaccionaron algunos candidatos al alza de la #inflación Variación anual del IPC llegó a 9,23% en abril, informó el @DANE_Colombia . Esto dijeron los aspirantes a la Presidencia sobre el dato, el más alto en 21 años. https://t.co/KkxlGRIPCi via @Portafolioco",negative,positive/neutral
41,"🇨🇴 ¿Qué pasaría en un ballotage entre @petrogustavo\r\ny @FicoGutierrez?\r\n\r\n🔍 A un mes de las elecciones que definirán el futuro del país, le preguntamos a 3 mil colombianos y colombianas y este es el resultado 👇\r\n\r\nhttps://t.co/Gsqqol98zk https://t.co/QDX164ZE9U",positive/neutral,negative
45,"@GustavoBolivar Debieran inventar algo diferente están igual que Chávez y Maduro cada vez que vienen elecciones inventaba guerras con Colombia, que había intereses en matarlo y ustedes igualito inventan enfermedades, atentados etc, etc.",positive/neutral,negative
46,"A pocos días de las elecciones presidenciales\r\nlas Autodefensas Gaitanistas de Colombia se declaran en Paro Armado hasta el 9 de mayo. El claro desinterés social del Gobierno actual, evidenciado en la permisión de hostigamiento contra los Colombianos.",negative,positive/neutral
52,"Episodio 8 - En este nuevo episodio Alicia Eugenia Silva nos plantea una reflexión, en estas elecciones ¿el país sigue polarizado? o ¿nace un nuevo bipartidismo en Colombia? https://t.co/HYnK26DpNn via @YouTube",positive/neutral,negative
